In [15]:
import csv
from urllib.request import urlopen
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT 
import pandas as pd
import io
import requests
from datetime import datetime, timedelta
import time
from datetime import date

In [18]:
## *****NOTE -
## *****This Needs to be Executed only once of the COVID_19, if DB is not present/ postgres is freshly installed.

# Connecting to default "postgres" database and using it we create covid_19 database
db_connection = psycopg2.connect(host='127.0.0.1',dbname="postgres", user="postgres" , password="root")

db_connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT) 

cursor = db_connection.cursor()

# Creating "covid_19" database
#cursor.execute("CREATE DATABASE covid_19;")



In [19]:
#Once the covdid 19 DB is created we connect to "covid_19"

db_connection = psycopg2.connect(host='127.0.0.1',dbname="covid_19", user="postgres" , password="root")

cursor = db_connection.cursor()

db_connection.commit()

In [20]:
cursor.execute("DROP TABLE IF EXISTS covid_daily_report")
cursor.execute("DROP TABLE IF EXISTS covid_daily_report_us")
cursor.execute("DROP TABLE IF EXISTS covid_timeseries_report")

In [21]:
db_connection.commit()

In [22]:
cursor.execute("""CREATE TABLE covid_daily_report(
                province_state TEXT,
                country_region TEXT,
                last_updated timestamp,
                confirmed INTEGER,
                deaths INTEGER,
                recovered INTEGER,
                latitude FLOAT,
                longitude FLOAT);""")

cursor.execute("""CREATE TABLE covid_daily_report_us(
                province_state TEXT,
                country_region TEXT,
                last_updated TEXT,
                confirmed INTEGER,
                deaths INTEGER,
                recovered INTEGER,
                active INTEGER,
                people_tested INTEGER,
                people_hospitalized INTEGER,
                latitude FLOAT,
                longitude FLOAT);""")

cursor.execute("""CREATE TABLE covid_timeseries_report(
                province_state TEXT,
                country_region TEXT,
                latitude FLOAT,
                longitude FLOAT,
                report_date TEXT,
                total INTEGER,
                category TEXT);""")

In [16]:
db_connection.commit()

### Pulling COVID-19 Realtime Data ( Daily World Report, Daily US Report and Timeseries Report)

In [ ]:
while True:
    
    #**************** Start of Fetching Daily Report *********************#
    utc_datetime = datetime.utcnow()-timedelta(days=1)
    dt = utc_datetime.strftime("%m-%d-%Y")
    
    print("Pulling data from server - Daily Report")
    url="https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/"+dt+".csv"
    s=requests.get(url).content
    df=pd.read_csv(io.StringIO(s.decode('utf-8')))
    df = df.rename(columns={
        "Province_State": "province_state",
        "Country_Region": "country_region",
        "Last_Update": "last_updated",
        "Confirmed": "confirmed",
        "Deaths": "deaths",
        "Recovered": "recovered",
        "Lat": "latitude",
        "Long_": "longitude"
    })
    df.drop(columns=['FIPS', 'Admin2', 'Active', 'Combined_Key'])
    df = df.fillna('')
    df.to_csv('covid_19_report_'+dt+'.csv',sep=',', encoding='utf-8', index=False)
    
    # delete existing data in the table
    cursor.execute("DELETE FROM covid_daily_report")
    # Saving the changes
    db_connection.commit()
    
    print("Started inserting data to DB - Daily Report")
    # Inserting each row
    for row in df.iterrows():
        province_state = row[1]['province_state']
        country_region = row[1]['country_region']
        latitude = row[1]['latitude']
        longitude = row[1]['longitude']

        # Incase the 's is present in the string it adds an escape character i.e. ''
        if(province_state != ''):
            province_state = province_state.replace("'","''")
        if(country_region != ''):
            country_region = country_region.replace("'","''")
        if(latitude == ''):
            latitude = 0.0
        if(longitude == ''):
            longitude = 0.0
        

        query = """
        INSERT into covid_daily_report(province_state, country_region,
        last_updated, confirmed,deaths, recovered, latitude, longitude) 
        values('%s','%s',TIMESTAMP '%s',%s,%s,%s,%s,%s);
        """ % (province_state, country_region,
               datetime.fromisoformat(row[1]['last_updated']),
              row[1]['confirmed'], row[1]['deaths'], row[1]['recovered'],
              latitude, longitude)
        cursor.execute(query)
    
    print("Ended inserting data to DB - Daily Report")
    # Close the connection - Saving the changes
    db_connection.commit()
    
    #**************** End of Fetching Daily Report *********************#
    
    
    #**************** Start of Fetching US Daily Report *********************#
    utc_datetime = datetime.utcnow()-timedelta(days=1)
    dt = utc_datetime.strftime("%m-%d-%Y")

    print("Pulling data from server - US Daily Report")
    url="https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports_us/"+dt+".csv"
    s=requests.get(url).content
    df=pd.read_csv(io.StringIO(s.decode('utf-8')))
    df = df.rename(columns={
        "Province_State": "province_state",
        "Country_Region": "country_region",
        "Last_Update": "last_updated",
        "Confirmed": "confirmed",
        "Deaths": "deaths",
        "Recovered": "recovered",
        "Active": "active",
        "People_Tested": "people_tested",
        "People_Hospitalized": "people_hospitalized",
        "Lat": "latitude",
        "Long_": "longitude"
    })
    df.drop(columns=['FIPS', 'Incident_Rate', 'Mortality_Rate', 'UID', 'ISO3', 'Testing_Rate', 'Hospitalization_Rate'])
    df = df.fillna('')
    df.to_csv('covid_19_report_us_'+dt+'.csv',sep=',', encoding='utf-8', index=False)

    # delete existing data in the table
    cursor.execute("DELETE FROM covid_daily_report_us")
    # Saving the changes
    db_connection.commit()

    print("Started inserting data to DB - US Daily Report")
    # Inserting each row
    for row in df.iterrows():
        province_state = row[1]['province_state']
        country_region = row[1]['country_region']
        latitude = row[1]['latitude']
        longitude = row[1]['longitude']
        confirmed = row[1]['confirmed']
        deaths = row[1]['deaths']
        recovered = row[1]['recovered']
        active = row[1]['active']
        people_tested = row[1]['people_tested']
        people_hospitalized = row[1]['people_hospitalized']

        # Incase the 's is present in the string it adds an escape character i.e. ''
        if(province_state != ''):
            province_state = province_state.replace("'","''")
        if(country_region != ''):
            country_region = country_region.replace("'","''")
        if(latitude == ''):
            latitude = 0.0
        if(longitude == ''):
            longitude = 0.0
        if(confirmed == ''):
            confirmed = 0
        if(deaths == ''):
            deaths = 0
        if(recovered == ''):
            recovered = 0
        if(active == ''):
            active = 0
        if(people_tested == ''):
            people_tested = 0
        if(people_hospitalized == ''):
            people_hospitalized = 0



        query = """
        INSERT into covid_daily_report_us(province_state, country_region,
        last_updated, confirmed,deaths, recovered,active,
        people_tested,people_hospitalized,latitude, longitude) 
        values('%s','%s','%s',%s,%s,%s,%s,%s,%s,%s,%s);
        """ % (province_state, country_region,
               row[1]['last_updated'],
              confirmed, deaths, recovered,active,
              people_tested,people_hospitalized,
              latitude, longitude)
        cursor.execute(query)

    print("Ended inserting data to DB - US Daily Report")
    # Close the connection - Saving the changes
    db_connection.commit()

    #**************** End of Fetching US Daily Report *********************#
    
    
    #**************** Start of Fetching Timeseries Report *********************#
    
     # delete existing data in the table
    cursor.execute("DELETE FROM covid_timeseries_report")
    # Saving the changes
    db_connection.commit()
    category = ['confirmed','deaths','recovered']
    for cat in category:
        print("Pulling data from server - " + cat)
            #"https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv
        url="https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_"+cat+"_global.csv"
        s=requests.get(url).content
        df_conf=pd.read_csv(io.StringIO(s.decode('utf-8')))
        df_conf = df_conf.rename(columns={
            "Province/State": "province_state",
            "Country/Region": "country_region",
            "Lat": "latitude",
            "Long": "longitude"
        })
        df_conf = df_conf.fillna('')
        
        df_conf.to_csv('time_series_19-covid-'+cat+'.csv',sep=',', encoding='utf-8', index=False)
        
        # Getting all the dates as list
        dates_col = df_conf.columns[4:len(df_conf.columns)]

        df_datewise= df_conf[['province_state', 'country_region', 'latitude', 'longitude']].copy()

        print("Started inserting data to DB - " + cat)
        for d in dates_col:
            lst_count = list(df_conf[[d][0]])
            lst_same_date = [list(df_conf[[d]])[0] for i in range(len(lst_count))]
            lst_data_type = [cat for i in range(len(lst_count))]
            df_datewise['report_date'] = lst_same_date
            df_datewise['total'] = lst_count
            df_datewise['category'] = lst_data_type

            # Inserting data to database datewise 
            # Inserting each row
            for row in df_datewise.iterrows():
                #print(row[1])
                province_state = row[1]['province_state']
                country_region = row[1]['country_region']

                # Incase the 's is present in the string it adds an escape character i.e. ''
                if(province_state != ''):
                    province_state = province_state.replace("'","''")
                if(country_region != ''):
                    country_region = country_region.replace("'","''")

                query = """
                INSERT into covid_timeseries_report(province_state, 
                    country_region, latitude, longitude,
                    report_date, total, category) 
                values('%s','%s',%s,%s,'%s',%s,'%s');
                """ % (province_state, country_region,
                       row[1]['latitude'], row[1]['longitude'],
                      row[1]['report_date'], row[1]['total'], row[1]['category'])
                cursor.execute(query)
        print("Ended inserting data to DB - " + cat)

        db_connection.commit()
    
    #**************** End of Fetching Timeseries Report *********************#
    
    # Sleep for 1 day; COVID19 data gets updated on daily basis
    now = datetime.now()
    timedate = ( (str(now.now())).split('.')[0]).split(' ')[0] + ' ' + ((str(now.now())).split('.')[0]).split(' ')[1]
 
    print(timedate, ' : Fetched data from GIT for Covid Daily Report and Going to sleep for 1-day now ...')
    time.sleep(24*60*60)    
    continue


Pulling data from server - Daily Report
Started inserting data to DB - Daily Report
Ended inserting data to DB - Daily Report
Pulling data from server - US Daily Report
Started inserting data to DB - US Daily Report
Ended inserting data to DB - US Daily Report
Pulling data from server - confirmed
Started inserting data to DB - confirmed
Ended inserting data to DB - confirmed
Pulling data from server - deaths
Started inserting data to DB - deaths
Ended inserting data to DB - deaths
Pulling data from server - recovered
Started inserting data to DB - recovered
Ended inserting data to DB - recovered
2020-04-15 21:13:23  : Fetched data from GIT for Covid Daily Report and Going to sleep for 1-day now ...
